In [1]:
import sys
sys.path.insert(0,"../../Code/Classes")
from DataClass import DataLoader

import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
def split_image(img,n_splits):
    #Assuming squares
    sz_xy = img.shape[0]
    sz_sm = int(sz_xy/n_splits)
    lay = img.shape[2]
    out = np.zeros((n_splits**2,sz_sm,sz_sm,lay))
    for i in range(n_splits):
        for j in range(n_splits):
            c_x = sz_sm*i
            c_y = sz_sm*j
            out[n_splits*i+j,:,:,:] = img[c_x:c_x+sz_sm,c_y:c_y+sz_sm,:]
    return out

def unsplit_image(imgs):
    num_splits = imgs.shape[0]
    s_splits = int(np.sqrt(num_splits))
    sm_res = imgs.shape[1]
    channels = imgs.shape[3]
    
    big_res = sm_res*s_splits
    
    out = np.zeros((big_res,big_res,channels))
    for i in range(s_splits):
        for j in range(s_splits):
            c_x = sm_res*i
            c_y = sm_res*j
            out[c_x:c_x+sm_res,c_y:c_y+sm_res,:]=imgs[s_splits*i+j,:,:,:]
    return out            

In [3]:
dataloader = DataLoader()
dataloader.load_std_folder("../../../Data/STD_Folder")
dataloader.parse_biclass()

In [4]:
size_f = 4096
size_r = 16
#Assuming square
splits = int(size_f/size_r)

In [5]:
full_num_img = len(dataloader.classfiles)
im_indices = [a for a in range(full_num_img)]

In [6]:
from sklearn.model_selection import train_test_split

samp_per_img = 256

split_range = [a for a in range(splits*splits)]
samp = train_test_split(split_range,train_size = samp_per_img)[0]

X = np.zeros((full_num_img*samp_per_img,size_r,size_r,3))
y = np.zeros((full_num_img*samp_per_img,size_r,size_r),bool)
for a in im_indices:
    c_ind = a*samp_per_img
    tmp = dataloader.get_data("flat",[a])[0]
    c_img = split_image(tmp,splits)
    c_class_tmp = dataloader.get_data("class",[a])[0]
    c_class = split_image(cv2.resize(c_class_tmp,(size_f,size_f)),splits)[:,:,:,0].astype(bool)
    X[c_ind:c_ind+samp_per_img,:,:,:] = c_img[samp]/255
    y[c_ind:c_ind+samp_per_img,:,:] = c_class[samp]
y = np.moveaxis(np.asarray([y],bool),(0,1,2,3),(3,0,1,2))

In [7]:
#Checking class distribution around 0.5
np.sum(y)/y.size

0.5235983933074565

In [8]:
#Splitting into train and test data
test_size = 0.1
val_size = 0.1

shape_range = [a for a in range(X.shape[0])]

t_ind,test_ind = train_test_split(shape_range,test_size=test_size)
train_ind,val_ind = train_test_split(t_ind,test_size=val_size)

X_train,y_train = X[train_ind],y[train_ind]
X_val,y_val = X[val_ind],y[val_ind]
X_test,y_test = X[test_ind],y[test_ind]

In [13]:
#16x16x1 -> 4x4x8

from tensorflow import keras

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=32,kernel_size=7,input_shape = [size_r,size_r,3],
                              activation="tanh", padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.MaxPool2D(2, padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.MaxPool2D(2, padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=8,kernel_size=3,activation="relu", padding="same"))

#model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
#model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=32,kernel_size=2,activation="relu", padding="same"))
model.add(keras.layers.Conv2D(1,kernel_size=3,activation="sigmoid", padding="same"))

In [14]:
model.compile(loss="mean_squared_error",metrics = [keras.metrics.RootMeanSquaredError()],
             optimizer="nadam")

In [15]:
history = model.fit(X_train,X_train,epochs=100,validation_data=(X_val,X_val),batch_size=32)

Epoch 1/100
512/512 [==============================] - 11s 19ms/step - loss: 0.0244 - root_mean_squared_error: 0.1563 - val_loss: 0.0115 - val_root_mean_squared_error: 0.1074
Epoch 2/100
512/512 [==============================] - 10s 19ms/step - loss: 0.0095 - root_mean_squared_error: 0.0975 - val_loss: 0.0083 - val_root_mean_squared_error: 0.0912
Epoch 3/100
512/512 [==============================] - 10s 19ms/step - loss: 0.0072 - root_mean_squared_error: 0.0846 - val_loss: 0.0072 - val_root_mean_squared_error: 0.0850
Epoch 4/100
512/512 [==============================] - 10s 19ms/step - loss: 0.0065 - root_mean_squared_error: 0.0806 - val_loss: 0.0065 - val_root_mean_squared_error: 0.0808
Epoch 5/100
512/512 [==============================] - 10s 19ms/step - loss: 0.0058 - root_mean_squared_error: 0.0761 - val_loss: 0.0054 - val_root_mean_squared_error: 0.0736
Epoch 6/100
512/512 [==============================] - 10s 19ms/step - loss: 0.0054 - root_mean_squared_error: 0.0737 - val_l

512/512 [==============================] - 10s 19ms/step - loss: 0.0040 - root_mean_squared_error: 0.0636 - val_loss: 0.0040 - val_root_mean_squared_error: 0.0634
Epoch 94/100
512/512 [==============================] - 10s 19ms/step - loss: 0.0041 - root_mean_squared_error: 0.0636 - val_loss: 0.0040 - val_root_mean_squared_error: 0.0632
Epoch 95/100
512/512 [==============================] - 10s 19ms/step - loss: 0.0040 - root_mean_squared_error: 0.0636 - val_loss: 0.0040 - val_root_mean_squared_error: 0.0631
Epoch 96/100
512/512 [==============================] - 10s 19ms/step - loss: 0.0040 - root_mean_squared_error: 0.0636 - val_loss: 0.0040 - val_root_mean_squared_error: 0.0635
Epoch 97/100
512/512 [==============================] - 10s 19ms/step - loss: 0.0040 - root_mean_squared_error: 0.0636 - val_loss: 0.0040 - val_root_mean_squared_error: 0.0634
Epoch 98/100
512/512 [==============================] - 10s 19ms/step - loss: 0.0040 - root_mean_squared_error: 0.0636 - val_loss: 0.

In [16]:
#Saving model
model.save("Flat_4x4x8_autoencoder.h5")

In [17]:
#Test error
np.sqrt(np.mean((model.predict(X_test)-X_test)**2))

0.06242532607676966

In [18]:
size_f = 4096
size_r = 32
#Assuming square
splits = int(size_f/size_r)

full_num_img = len(dataloader.classfiles)
im_indices = [a for a in range(full_num_img)]


from sklearn.model_selection import train_test_split

samp_per_img = 128

split_range = [a for a in range(splits*splits)]
samp = train_test_split(split_range,train_size = samp_per_img)[0]

X = np.zeros((full_num_img*samp_per_img,size_r,size_r,3))
y = np.zeros((full_num_img*samp_per_img,size_r,size_r),bool)
for a in im_indices:
    c_ind = a*samp_per_img
    tmp = dataloader.get_data("flat",[a])[0]
    c_img = split_image(tmp,splits)
    c_class_tmp = dataloader.get_data("class",[a])[0]
    c_class = split_image(cv2.resize(c_class_tmp,(size_f,size_f)),splits)[:,:,:,0].astype(bool)
    X[c_ind:c_ind+samp_per_img,:,:,:] = c_img[samp]/255
    y[c_ind:c_ind+samp_per_img,:,:] = c_class[samp]
y = np.moveaxis(np.asarray([y],bool),(0,1,2,3),(3,0,1,2))


#Splitting into train and test data
test_size = 0.1
val_size = 0.1

shape_range = [a for a in range(X.shape[0])]

t_ind,test_ind = train_test_split(shape_range,test_size=test_size)
train_ind,val_ind = train_test_split(t_ind,test_size=val_size)

X_train,y_train = X[train_ind],y[train_ind]
X_val,y_val = X[val_ind],y[val_ind]
X_test,y_test = X[test_ind],y[test_ind]

In [39]:
#32x32x1 -> 8x8x8

from tensorflow import keras

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=32,kernel_size=7,input_shape = [size_r,size_r,3],
                              activation="tanh", padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.MaxPool2D(2, padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.MaxPool2D(2, padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=8,kernel_size=3,activation="relu", padding="same"))

#model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
#model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
model.add(keras.layers.Conv2D(1,kernel_size=3,activation="sigmoid", padding="same"))

In [40]:
model.compile(loss="mean_squared_error",metrics = [keras.metrics.RootMeanSquaredError()],
             optimizer="nadam")

In [41]:
history = model.fit(X_train,X_train,epochs=100,validation_data=(X_val,X_val),batch_size=32)

Epoch 1/100
256/256 [==============================] - 16s 61ms/step - loss: 0.0289 - root_mean_squared_error: 0.1699 - val_loss: 0.0282 - val_root_mean_squared_error: 0.1680
Epoch 2/100
256/256 [==============================] - 15s 60ms/step - loss: 0.0293 - root_mean_squared_error: 0.1710 - val_loss: 0.0277 - val_root_mean_squared_error: 0.1664
Epoch 3/100
256/256 [==============================] - 15s 60ms/step - loss: 0.0234 - root_mean_squared_error: 0.1528 - val_loss: 0.0121 - val_root_mean_squared_error: 0.1098
Epoch 4/100
256/256 [==============================] - 15s 60ms/step - loss: 0.0109 - root_mean_squared_error: 0.1046 - val_loss: 0.0115 - val_root_mean_squared_error: 0.1072
Epoch 5/100
256/256 [==============================] - 16s 63ms/step - loss: 0.0082 - root_mean_squared_error: 0.0908 - val_loss: 0.0090 - val_root_mean_squared_error: 0.0946
Epoch 6/100
256/256 [==============================] - 16s 62ms/step - loss: 0.0069 - root_mean_squared_error: 0.0831 - val_l

256/256 [==============================] - 15s 60ms/step - loss: 0.0042 - root_mean_squared_error: 0.0645 - val_loss: 0.0042 - val_root_mean_squared_error: 0.0648
Epoch 94/100
256/256 [==============================] - 15s 59ms/step - loss: 0.0042 - root_mean_squared_error: 0.0645 - val_loss: 0.0042 - val_root_mean_squared_error: 0.0645
Epoch 95/100
256/256 [==============================] - 15s 59ms/step - loss: 0.0042 - root_mean_squared_error: 0.0646 - val_loss: 0.0043 - val_root_mean_squared_error: 0.0655
Epoch 96/100
256/256 [==============================] - 15s 59ms/step - loss: 0.0041 - root_mean_squared_error: 0.0643 - val_loss: 0.0042 - val_root_mean_squared_error: 0.0648
Epoch 97/100
256/256 [==============================] - 15s 59ms/step - loss: 0.0042 - root_mean_squared_error: 0.0645 - val_loss: 0.0042 - val_root_mean_squared_error: 0.0645
Epoch 98/100
256/256 [==============================] - 15s 59ms/step - loss: 0.0041 - root_mean_squared_error: 0.0644 - val_loss: 0.

In [42]:
#Saving model
#Saving model
model.save("Flat_8x8x8_autoencoder.h5")

In [43]:
size_f = 4096
size_r = 64
#Assuming square
splits = int(size_f/size_r)

full_num_img = len(dataloader.classfiles)
im_indices = [a for a in range(full_num_img)]


from sklearn.model_selection import train_test_split

samp_per_img = 64

split_range = [a for a in range(splits*splits)]
samp = train_test_split(split_range,train_size = samp_per_img)[0]

X = np.zeros((full_num_img*samp_per_img,size_r,size_r,3))
y = np.zeros((full_num_img*samp_per_img,size_r,size_r),bool)
for a in im_indices:
    c_ind = a*samp_per_img
    tmp = dataloader.get_data("flat",[a])[0]
    c_img = split_image(tmp,splits)
    c_class_tmp = dataloader.get_data("class",[a])[0]
    c_class = split_image(cv2.resize(c_class_tmp,(size_f,size_f)),splits)[:,:,:,0].astype(bool)
    X[c_ind:c_ind+samp_per_img,:,:,:] = c_img[samp]/255
    y[c_ind:c_ind+samp_per_img,:,:] = c_class[samp]
y = np.moveaxis(np.asarray([y],bool),(0,1,2,3),(3,0,1,2))


#Splitting into train and test data
test_size = 0.1
val_size = 0.1

shape_range = [a for a in range(X.shape[0])]

t_ind,test_ind = train_test_split(shape_range,test_size=test_size)
train_ind,val_ind = train_test_split(t_ind,test_size=val_size)

X_train,y_train = X[train_ind],y[train_ind]
X_val,y_val = X[val_ind],y[val_ind]
X_test,y_test = X[test_ind],y[test_ind]

In [50]:
from tensorflow import keras

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=32,kernel_size=7,input_shape = [size_r,size_r,3],
                              activation="tanh", padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.MaxPool2D(2, padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.MaxPool2D(2, padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=8,kernel_size=3,activation="relu", padding="same"))

#model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
#model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
model.add(keras.layers.Conv2D(1,kernel_size=3,activation="sigmoid", padding="same"))

model.compile(loss="mean_squared_error",metrics = [keras.metrics.RootMeanSquaredError()],
             optimizer="nadam")

In [51]:
history = model.fit(X_train,X_train,epochs=100,validation_data=(X_val,X_val),batch_size=32)

Epoch 1/100
128/128 [==============================] - 29s 221ms/step - loss: 0.0278 - root_mean_squared_error: 0.1668 - val_loss: 0.0255 - val_root_mean_squared_error: 0.1597
Epoch 2/100
128/128 [==============================] - 28s 221ms/step - loss: 0.0243 - root_mean_squared_error: 0.1557 - val_loss: 0.0162 - val_root_mean_squared_error: 0.1271
Epoch 3/100
128/128 [==============================] - 28s 220ms/step - loss: 0.0154 - root_mean_squared_error: 0.1242 - val_loss: 0.0112 - val_root_mean_squared_error: 0.1060
Epoch 4/100
128/128 [==============================] - 29s 228ms/step - loss: 0.0127 - root_mean_squared_error: 0.1129 - val_loss: 0.0120 - val_root_mean_squared_error: 0.1094
Epoch 5/100
128/128 [==============================] - 30s 237ms/step - loss: 0.0110 - root_mean_squared_error: 0.1050 - val_loss: 0.0087 - val_root_mean_squared_error: 0.0934
Epoch 6/100
128/128 [==============================] - 28s 223ms/step - loss: 0.0100 - root_mean_squared_error: 0.0999 -

KeyboardInterrupt: 

In [52]:
model.save("Flat_16x16x8_autoencoder.h5")

In [4]:
size_f = 4096
size_r = 128
#Assuming square
splits = int(size_f/size_r)

full_num_img = len(dataloader.classfiles)
im_indices = [a for a in range(full_num_img)]


from sklearn.model_selection import train_test_split

samp_per_img = 64

split_range = [a for a in range(splits*splits)]
samp = train_test_split(split_range,train_size = samp_per_img)[0]

X = np.zeros((full_num_img*samp_per_img,size_r,size_r,3))
y = np.zeros((full_num_img*samp_per_img,size_r,size_r),bool)
for a in im_indices:
    c_ind = a*samp_per_img
    tmp = dataloader.get_data("flat",[a])[0]
    c_img = split_image(tmp,splits)
    c_class_tmp = dataloader.get_data("class",[a])[0]
    c_class = split_image(cv2.resize(c_class_tmp,(size_f,size_f)),splits)[:,:,:,0].astype(bool)
    X[c_ind:c_ind+samp_per_img,:,:,:] = c_img[samp]/255
    y[c_ind:c_ind+samp_per_img,:,:] = c_class[samp]
y = np.moveaxis(np.asarray([y],bool),(0,1,2,3),(3,0,1,2))


#Splitting into train and test data
test_size = 0.1
val_size = 0.1

shape_range = [a for a in range(X.shape[0])]

t_ind,test_ind = train_test_split(shape_range,test_size=test_size)
train_ind,val_ind = train_test_split(t_ind,test_size=val_size)

X_train,y_train = X[train_ind],y[train_ind]
X_val,y_val = X[val_ind],y[val_ind]
X_test,y_test = X[test_ind],y[test_ind]

In [54]:
del model

In [5]:
from tensorflow import keras

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=32,kernel_size=7,input_shape = [size_r,size_r,3],
                              activation="tanh", padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.MaxPool2D(2, padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.MaxPool2D(2, padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=8,kernel_size=3,activation="relu", padding="same"))

#model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
#model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
model.add(keras.layers.Conv2D(1,kernel_size=3,activation="sigmoid", padding="same"))

model.compile(loss="mean_squared_error",metrics = [keras.metrics.RootMeanSquaredError()],
             optimizer="nadam")

In [6]:
history = model.fit(X_train,X_train,epochs=100,validation_data=(X_val,X_val),batch_size=32)

Epoch 1/100
128/128 [==============================] - 128s 992ms/step - loss: 0.0284 - root_mean_squared_error: 0.1686 - val_loss: 0.0295 - val_root_mean_squared_error: 0.1716
Epoch 2/100
128/128 [==============================] - 126s 982ms/step - loss: 0.0287 - root_mean_squared_error: 0.1695 - val_loss: 0.0288 - val_root_mean_squared_error: 0.1698
Epoch 3/100
128/128 [==============================] - 124s 967ms/step - loss: 0.0281 - root_mean_squared_error: 0.1675 - val_loss: 0.0286 - val_root_mean_squared_error: 0.1692
Epoch 4/100
128/128 [==============================] - 124s 968ms/step - loss: 0.0276 - root_mean_squared_error: 0.1661 - val_loss: 0.0273 - val_root_mean_squared_error: 0.1652
Epoch 5/100
128/128 [==============================] - 124s 970ms/step - loss: 0.0281 - root_mean_squared_error: 0.1676 - val_loss: 0.0278 - val_root_mean_squared_error: 0.1668
Epoch 6/100
128/128 [==============================] - 124s 970ms/step - loss: 0.0266 - root_mean_squared_error: 0.

KeyboardInterrupt: 

In [7]:
model.save("Flat_32x32x8_autoencoder.h5")

In [7]:
del model